In [1]:
!pip install -q transformers einops accelerate langchain bitsandbytes


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install langchain


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!nvidia-smi

Mon Jun 12 16:12:40 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 531.61                 Driver Version: 531.61       CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                      TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3060       WDDM | 00000000:0A:00.0  On |                  N/A |
|  0%   49C    P5               16W / 170W|  12085MiB / 12288MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer, pipeline
import torch

model = "tiiuae/falcon-7b-instruct" #tiiuae/falcon-40b-instruct

tokenizer = AutoTokenizer.from_pretrained(model)

pipeline = pipeline(
    "text-generation", #task
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    temperature=0.1,
    device_map="auto",
    max_length=1000,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
)


# LLM = llm = HuggingFacePipeline.from_model_id(
#     model_id=model,
#     task="text-generation",
#     model_kwargs={"temperature": 0.1, "max_length": 1000,"trust_remote_code": True},
# )

Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
Loading checkpoint shards:   0%|                                                                 | 0/2 [00:00<?, ?it/s]

In [2]:
llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature':0.1})

In [19]:
from langchain import PromptTemplate,  LLMChain

template = """Answer the question as truthfully as possible using the provided text, and if the answer is not contained within the text below, say "I don't know"

Context:
{context}

{query}""".strip()

context = '''Ezio Auditore da Firenze (Italian pronunciation: [ˈɛttsjo audiˈtoːre da (f)fiˈrɛntse]) is a fictional character in the video game series Assassin's Creed, an Italian master assassin who serves as the protagonist of the series' games set during the Italian Renaissance. His life and career as an assassin are chronicled in Assassin's Creed II, II: Discovery, Brotherhood, and Revelations, and the short films Assassin's Creed: Lineage and Assassin's Creed Embers. All games (excluding II: Discovery) and films he appears in were re-released as an enhanced bundle, The Ezio Collection, in 2016. Ezio has also been frequently referenced or made smaller appearances in other media within the franchise. Actor Roger Craig Smith has consistently provided the character's voice throughout his appearances, while Devon Bostick portrayed him in live-action in Lineage.

Within the series' alternate historical setting, Ezio was born into Italian nobility from Florence in the year 1459. His family had long been loyal to the Assassin Brotherhood, a fictional organization inspired by the real-life Order of Assassins dedicated to protecting peace and freedom, but Ezio did not learn about his Assassin heritage until his late teens, after most of his immediate kin were killed during the Pazzi conspiracy. His quest to track down those responsible for killing his family eventually sets him up against the villainous Templar Order led by the House of Borgia. Spending years to fight against Rodrigo and Cesare Borgia and their henchmen, he eventually re-establishes the Brotherhood as the dominant force in Italy. His further adventures lead him to Spain and the Ottoman Empire, where he is also essential in overcoming Templar threats and restoring the Assassins. After his retirement from the Brotherhood, he lives a peaceful life in rural Tuscany until his eventual death from a heart attack in 1524.

The character has received critical acclaim and is often named among the greatest video game characters of all time. While most of his praise focuses on his portrayal and growth throughout the series, as well as the unique chronicling of his entire life, he has also been noted as one of the most attractive video game characters of all time. Due to his reception and the fact that he is the only character in the series who is the protagonist of multiple major installments of the franchise,[a] he is usually considered the face of the franchise and its most popular character. Ezio's popularity has led to several crossover appearances outside of the Assassin's Creed series, notably Soulcalibur V, Fortnite, and Brawlhalla, as a guest character."'''

prompt_template = PromptTemplate(
    input_variables=["query", "context"],
    template=template
)

query = "Give summary of above context in 2 sentences"

prompt = prompt_template.format(query=query, context=context)

# llm_chain = LLMChain(prompt=prompt, llm=llm)

llm(prompt)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


":\nEzio Auditore da Firenze is a fictional character in the Assassin's Creed series, an Italian master assassin who serves as the protagonist of the series' games set during the Italian Renaissance. His life and career as an assassin are chronicled in Assassin's Creed II, II: Discovery, Brotherhood, and Revelations, and the short films Assassin's Creed: Lineage and Assassin's Creed Embers."

In [17]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate

In [18]:
# How you would like your reponse structured. This is basically a fancy prompt template
response_schemas = [
    ResponseSchema(name="What", description="What are the risks?"),
]

# How you would like to parse your output
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [19]:
format_instructions = output_parser.get_format_instructions()
print (format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "\`\`\`json" and "\`\`\`":

```json
{
	"What": string  // What are the risks?
}
```


In [20]:
template2 = """
    Data: {user_input}
    Question:{input_questions}
"""

In [21]:
input_text = '''Risk of favoring or disadvantaging customers in offering or servicing financial products and services, which includes treating customers unfairly by engaging in unfair, deceptive, or
abusive acts or practices. For example, using improper means to induce a customer to take an action the customer may not otherwise have taken like selling products/services or
settling debt, or taking advantage of customer's trust in the organization or lack of knowledge of product/service. This also includes the risk of wrongly denying or limiting the
options available to the customers.'''

input_questions = '''What are the Risks?'''

In [27]:
prompt2 = PromptTemplate(
    template=template2,
    input_variables=["user_input"],
    partial_variables={"input_questions": input_questions}
)

promptValue = prompt2.format(user_input=input_text)

print(prompt2)

input_variables=['user_input'] output_parser=None partial_variables={'input_questions': 'What are the Risks?'} template='\n    Data: {user_input}\n    Question:{input_questions}\n' template_format='f-string' validate_template=True


In [28]:
llm_chain2 = LLMChain(prompt=prompt2, llm=llm)

In [29]:
llm_output = llm_chain2.run(input_text)
llm_output

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


"The Risks include:\n1. Financial Loss: This risk includes the risk of loss of money and/or property in connection with an activity. Examples may include, but are not limited to: fraudulent or wrongful\nactions by employees, or fraudulent transactions by customers.\n2. Regulatory Risk: This risk includes the risk of regulatory enforcement actions, including penalties, fines and other costs. This risk can result from the\norganization's actions or lack thereof in regulatory compliance. For example:\na. Non-compliance of data security regulations;\nb. Non-compliance of licensing and/or regulatory regulations\n3. Operational Risk: This risk includes the risk of errors, fraud or other events arising from or related to the organization's business operations.\nExamples may include the risk of inadequate staffing of staff or employees or inadequate risk management systems in use.\n4. Credit Risk: This risk includes the risk of loss or damage to reputation or creditworthiness of the organizati

In [30]:
# output_parser.parse(llm_output)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ C:\Users\Rahul                                                                                   │
│ Sinha\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain\output_parsers\json.py │
│ :24 in parse_and_check_json_markdown                                                             │
│                                                                                                  │
│   21                                                                                             │
│   22 def parse_and_check_json_markdown(text: str, expected_keys: List[str]) -> dict:             │
│   23 │   try:                                                                                    │
│ ❱ 24 │   │   json_obj = parse_json_markdown(text)                                                │
│   25 │   except json.JSONDecodeError as e:                                                       │
│   26 │   │   raise OutputParserException(f"Got invalid JSON object. Error: {e}")                 │
│   27 │   for key in expected_keys:                                                               │
│                                                                                                  │
│ C:\Users\Rahul                                                                                   │
│ Sinha\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain\output_parsers\json.py │
│ :17 in parse_json_markdown                                                                       │
│                                                                                                  │
│   14 │   json_string = json_string.strip()                                                       │
│   15 │                                                                                           │
│   16 │   # Parse the JSON string into a Python dictionary                                        │
│ ❱ 17 │   parsed = json.loads(json_string)                                                        │
│   18 │                                                                                           │
│   19 │   return parsed                                                                           │
│   20                                                                                             │
│                                                                                                  │
│ C:\Users\Rahul Sinha\AppData\Local\Programs\Python\Python311\Lib\json\__init__.py:346 in loads   │
│                                                                                                  │
│   343 │   if (cls is None and object_hook is None and                                            │
│   344 │   │   │   parse_int is None and parse_float is None and                                  │
│   345 │   │   │   parse_constant is None and object_pairs_hook is None and not kw):              │
│ ❱ 346 │   │   return _default_decoder.decode(s)                                                  │
│   347 │   if cls is None:                                                                        │
│   348 │   │   cls = JSONDecoder                                                                  │
│   349 │   if object_hook is not None:                                                            │
│                                                                                                  │
│ C:\Users\Rahul Sinha\AppData\Local\Programs\Python\Python311\Lib\json\decoder.py:337 in decode   │
│                                                                                                  │
│   334 │   │   containing a JSON document).                                                       │
│   335 │   │                                                                                      │
│   336 │   │   """                                          